In [1]:
#this jupiter notebook rassemble my work in order
#first we import every libraries that we need
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import json
import requests
import numpy as np
from concurrent.futures import ThreadPoolExecutor
#then we create the file were the datas are gonna go
test = f"./fulldatas.json"
#we initiate the driver for the selenium part
def driver_gen():
    driver = webdriver.Chrome(executable_path="driver/chromedriver")
    return driver

In [30]:
# we get data from the list of links func 
def clean_data(listed):
    house_number = 1
    dict_of_informations= {}
    list_of_link = listed
    for lk in list_of_link : 
        r = requests.get(lk)
        content = r.content
        property_details_page = BeautifulSoup(content,"html.parser")
        dict_of_data= {}
        #we clean the datas by finding tags
        for tr in property_details_page.find_all('tr', attrs={'class':'classified-table__row'}):
            #these are the tage were our infos are
            value = tr.find('th')
            data = tr.find('td')
            if value != None :
                #we clean them using regex yay !
                title = re.sub(r'\s*<[^>]*>\s*','',str(value))
                if data != None : 
                    datas = re.sub(r'\s*<[^>]*>\s*','',str(data))
                    # we delete the big space to replace them with 1 only
                    datass = " ".join(datas.split())
                    #we store data in the dict
                    dict_of_data[title] = datass
                else: 
                    #if there are no datas we write none
                    dict_of_data[title] = None
            else: 
                continue
        #we put the datas in a another dict with all the house listed
        dict_of_informations[house_number] = dict_of_data
        print(dict_of_informations[house_number]['Address'])
        addres = dict_of_informations[house_number]['Address']
        infos = list(addres.split(" "))
        print(infos[3])
        
        house_number += 1 

    return dict_of_informations
    #right now the data is are dictionaries in a dictionary

In [12]:
# we list all the links to scrape them after
def get_links_and_clean(num, driver):
    list_of_link = []
    #still doing by page number is gonna change

    page_num = str(num) + "&orderBy=relevance"
    url = (
            "https://www.immoweb.be/en/search/house-and-apartment/for-sale?countries=BE&page=" + page_num
            )
    driver.get(url)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    dict_of_informations = {}
    for link in soup.find_all("a", class_="card__title-link") : 
        list_of_link.append(link['href'])
    driver.quit()
    #calling clean function to clean and return clean data
    #not the good way i think incorporate links finding in data cleaning
    
    return list_of_link


In [13]:
links = get_links_and_clean(1,driver_gen())

C:\Users\feldm\AppData\Local\Temp\ipykernel_37636\1950009225.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="driver/chromedriver")


In [31]:
clean_data(links)

Brusselsesteenweg 6909050 - Gentbrugge
Gentbrugge
Brusselsesteenweg 6909050 - Gentbrugge
Gentbrugge
Brusselsesteenweg 6909050 - Gentbrugge
Gentbrugge
Sint-Denijslaan 19000 - Gent
Gent
Sint-Denijslaan 19000 - Gent
Gent
Sint-Denijslaan 19000 - Gent
Gent
Sint-Denijslaan 19000 - Gent
Gent
Sint-Denijslaan 19000 - Gent
Gent
Sint-Denijslaan 19000 - Gent
Gent
Sint-Denijslaan 19000 - Gent
Gent
Sint-Denijslaan 19000 - Gent
Gent
Sint-Denijslaan 19000 - Gent
Gent
Sint-Denijslaan 19000 - Gent
Gent
Sint-Denijslaan 19000 - Gent
Gent
Sint-Denijslaan 19000 - Gent
Gent
Sint-Denijslaan 19000 - Gent
Gent
Sint-Denijslaan 19000 - Gent
Gent
Sint-Denijslaan 19000 - Gent
Gent
Sint-Denijslaan 19000 - Gent
Gent
Sint-Denijslaan 19000 - Gent
Gent
Sint-Denijslaan 19000 - Gent
Gent
Avenue de la Charmille 221200 - Woluwe-Saint-Lambert
Charmille
Chausée de Bruxelles 135 A1310 - La Hulpe
135
Stationstraat 309600 - Ronse
Ronse
Rue Charles Magnette 24/404000 - Liège
24/404000
Brusselsesteenweg 6909050 - Gentbrugge
Gentbr

{1: {'Available date': 'September 1 2024 - 12:00 AM',
  'Construction year': '2022',
  'Floor': '4',
  'Number of floors': '6',
  'Building condition': 'As new',
  'Number of frontages': '4',
  'Covered parking spaces': '101',
  'Outdoor parking spaces': '10',
  'Surroundings type': 'Urban',
  'Living area': '96m²square meters',
  'Living room surface': '16m²square meters',
  'Dining room': 'Yes',
  'Kitchen type': 'Hyper equipped',
  'Kitchen surface': '7m²square meters',
  'Bedrooms': '2',
  'Bedroom 1 surface': '15m²square meters',
  'Bedroom 2 surface': '10m²square meters',
  'Dressing room': 'No',
  'Bathrooms': '1',
  'Toilets': '1',
  'Office': 'No',
  'Furnished': 'No',
  'Armored door': 'No',
  'Garden': 'Yes',
  'Garden orientation': 'North West',
  'Terrace surface': '16m²square meters',
  'Primary energy consumption': '1kWh/m²kilowatt hour per square meters',
  'Energy class': 'A',
  '': None,
  'Reference number of the EPC report': 'Not specified',
  'CO₂ emission': 'Not s

In [5]:
#saving the data in a json file
def saving_datas(datas):
    with open(test, 'a') as file:
        json.dump(str(datas), file, indent=4)

In [6]:
#creating of the thread pool
#need to return on this function to divide the selenium and requests tasks
# map 300 links to the 36*300 houses to the 8 or 16 drivers and threads
def pool():
    # 1 driver by thread (we have four thread here)
    drivers = [driver_gen() for _ in range(2)]
    #division of the number of pages by the number of threads 
    division = np.array_split(np.arange(1,4),2)
    #creation of a pool of threads
    with ThreadPoolExecutor(max_workers=2) as executor :
        #it is where it needs refinement
        results = executor.map(get_links_and_clean,division,drivers) 
        for result in results:
            saving_datas(result)
            print(result)

pool()


#bug = i get the same house over and over in my final dictionary 
#but all the links are different ... Why ? 

C:\Users\feldm\AppData\Local\Temp\ipykernel_37636\1950009225.py:14: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="driver/chromedriver")


KeyboardInterrupt: 